In [108]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time

# Multiresolution mesh editing: 
For this task, you will compute a mesh deformation based on the rotations and translations applied interactively to a subset of its vertices via the mouse. Let $H$ be the set of "handle" vertices that the user can manipulate (or leave fixed). We want to compute a deformation for the remaining vertices, denoted as $R$.

Let $\mathcal{S}$ be our input surface, represented as a triangle mesh. We want to compute a new surface that contains:
- the vertices in $H$ (the handles) translated/rotated using the user-provided transformation $t$, and
- the vertices in $R$ properly deformed using the algorithm described next.

The algorithm is divided in three phases:

1. removing high-frequency details,
2. deforming the smooth mesh, and
3. transferring high-frequency details to the deformed surface.

In [109]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
# v, f = igl.read_triangle_mesh('data/woody-lo.off')
# labels = np.load('data/woody-lo.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [110]:
print(labels.shape)
print(labels)
print(v.shape)

(14347,)
[1 1 1 ... 2 2 2]
(14347, 3)


## Selecting the handles

In [111]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6)) # saves transformations in a martix 
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    # print('slices', slices)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [112]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [113]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    handle_indices = np.where(labels!=0)[0]
    # smooth original mesh (except vertex handles)
    B = smooth_mesh(v, f, handle_indices, v)
    # smooth the target mesh (except vertex handles)
    B_prime =  smooth_mesh(v, f, handle_indices, target_pos)
    """ Encoding details of B """
    B_norms = igl.per_vertex_normals(B, f)
    adjacent_vs = igl.adjacency_list(f)
    detail_b = np.zeros_like(B)
    dets = v-B
    coeffs = np.zeros_like(dets) # coefficients of the reference frame
    tangent_vectors = np.zeros_like(B) # Store the actual tangent vector
    for idx, vertex in enumerate(B):
        # get the detail at this vertex
        di = dets[idx]
        # find the edge with the longest projection onto the tangent plane
        max_tangent_norm = 0
        best_tangent = np.zeros(3)
        for vert_idx in adjacent_vs[idx]:
            norm = B_norms[idx]
            edge = B[vert_idx] - vertex
            proj_normal = np.dot(edge, norm) * norm
            proj_tangent = edge - proj_normal
            tangent_norm = np.linalg.norm(proj_tangent)
            if tangent_norm > max_tangent_norm:
                max_tangent_norm = tangent_norm
                best_tangent = proj_tangent
        # get the vectors in our frame
        xi = best_tangent / np.linalg.norm(best_tangent) # normalized tangent
        # save the tangent for the B'
        tangent_vectors[idx] = xi # Store the tangent vector
        norm = B_norms[idx]
        yi = np.cross(norm, xi) # cross product
        # encode the detail -- details[idx] = di[0]*xi + di[1]*yi + di[2]*norm -> coeffs are di[0], di[1], di[2]
        coeffs[idx] = [np.dot(di, xi), np.dot(di, yi), np.dot(di, norm)]
            
    Bp_norms = igl.per_vertex_normals(B_prime, f)
    dis_p = np.zeros_like(dets)
    for idx, vertex in enumerate(B_prime):
        coeff = coeffs[idx] # coefficients of the reference frame
        norm = Bp_norms[idx]
        xi_prime = tangent_vectors[idx] # using the same tangents as prev
        yi_prime = np.cross(norm, xi_prime)
        dis_p[idx] = coeff[0]*xi_prime + coeff[1]*yi_prime + coeff[2]*norm

    S = B_prime + dis_p
    target_pos = S
    # add back details 
    return target_pos
pos_f.deformer = position_deformer
''' (Optional) Register this function to perform interactive deformation
pos_f.deformer = position_deformer
'''

' (Optional) Register this function to perform interactive deformation\npos_f.deformer = position_deformer\n'

In [114]:
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

## Step 1: Smooth base surface (removal of high-frequency details)
We remove the high-frequency details from the vertices $R$ in $\mathcal{S}$ by minimizing the thin-plate energy, which involves solving a bi-Laplacian system arising from the quadratic energy minimization:
<img align="center" width="200" src="https://i.imgur.com/6IRzdBj.png">

where $o_H$ are the handle $H$'s vertex positions, $L_\omega$ is the cotan Laplacian of $\mathcal{S}$, and $M$ is the mass matrix of $\mathcal{S}$.

Notice that $L_\omega$ is the symmetric matrix consisting of the cotangent weights ONLY (without the division by Voronoi areas). In other words, it evaluates an "integrated" Laplacian rather than an "averaged" laplacian when applied to a vector of vertices. The inverse mass matrix appearing in the formula above then applies the appropriate rescaling so that the laplacian operator can be applied again (i.e., so that the Laplacian value computed at each vertex can be interpreted as a piecewise linear scalar field whose Laplacian can be computed).
This optimization will produce a mesh similar to the one in Figure 2. Note that the part of the surface that we want to deform is now free of high-frequency details. We call this mesh $\mathcal{B}$.


*Relevant `scipy` functions:* `scipy.sparse.csc_matrix`, `scipy.sparse.diags`, 

<!-- # Laplacian smoothing = doing explicit smoothung  -->
<!-- -- compute new pos of vertex as pos of vertex + displacement * laplacian  -->

In [100]:
from scipy.sparse.linalg import inv, spsolve

def smooth_mesh(v_in: np.ndarray, f_in: np.ndarray, handle_indices: np.ndarray, v_new: np.ndarray) -> np.ndarray:
    """ Function to smooth the mesh by solivng a bilaplacian minimization w.r.t old positions of handle.
        v_new is the desired position of the vertices after the transformation 
            (i.e. v if just the first smoothing step, or the transformed position of the handle if the deform smoothing step)
      """
    # Compute delta (the differential coordinates) for each vertex using the Laplacian: 
    M = igl.massmatrix(v_in, f_in, igl.MASSMATRIX_TYPE_VORONOI)
    M_inv = inv(M)
    S = igl.cotmatrix(v_in, f_in)
    L = M_inv @ S
    delta = L @ v_in 
    
    # Do the minimization: solving for x' in Ax' = b, where x' is the new vertex positions

    A = 2 * S @ M_inv @ S  # A = 2L_w M^-1 L_w 
    b_big = 2 * S @ delta # b = 2L_w * d -- these are the constraints on b

    # b is 0 if vertex is not a handle, and b is the delta if it is a handle
    b = np.zeros_like(v_in)
    for idx in handle_indices: 
        b[idx] = b_big[idx]

    # Break up A into 4 parts, and v into 2 to solve for unconstrained vertices 
    """ Split A into 4 parts: 
            AFF = unconstrained  x unconstrained  -- what we are solving for
            AFC = unconstrained  x constrained   
            ACF = constrained    x unconstrained  
            ACC = constrained    x constrained  -- we don't use because we need to solve for free
        Split v into 2 parts:
            x_f = unconstrained 
            x_c = constrained 
    """

    constraint_mask = np.zeros(v_in.shape[0], dtype=bool) 
    constraint_mask[handle_indices] = True
    unconstrained_v = np.where(~constraint_mask)[0] # array of indices of unconstrained verts 
    # print(unconstrained_v)
    constrained_v = np.where(constraint_mask)[0] # indices of constraint vertices
    # print(constrained_v)

    A_ff = A[unconstrained_v][:, unconstrained_v] # unconstrained rows x unconstrained columns
    A_fc = A[unconstrained_v][:, constrained_v] # unconstrained rows x constrained columns
    x_c = v_new[constrained_v] # constrained vertices have fixed positions (either the old pos of handle, v, or the transformed position)

    # solve A_ff x_f = b - A_fc x_c
    rhs = b[unconstrained_v] - A_fc @ x_c # b - A_fc vc
    x_f = spsolve(A_ff, rhs) # Aff*vf = rhs

    # now stack the unconstrained and constrained vertices together
    x = np.zeros_like(v_in)
    x[unconstrained_v] = x_f
    x[constrained_v] = x_c # constrained vertices are fixed

    return x

In [ ]:
handle_idx = np.where(labels != 0)[0] # indices of handle vertices
x = smooth_mesh(v,f,handle_idx, v) # smooth the mesh using the handle vertices

In [ ]:
new_pos = np.copy(v)
handle1_idx = np.where(labels == 1)[0] # indices of handle vertices
for i in range(len(handle1_idx)):
    new_pos[handle1_idx[i]] = v[handle1_idx[i]]+np.array([0,0.5,0])

[   0    1    2    3    4    5    6    7    8    9   10   11   12   13
   14   15   16   17   18   19   20   21   22   23   24   25   26   27
   28   29   30   31   32   33   34   35   36   37   38   39   40   41
   42   43   44   45   46   47   48   49   50   51   52   53   54   55
   56   57   58   59   60   61   62   63   64   65   66   67   68   69
   70   71   72   73   74   75   76   77   78   79   80   81   82   83
   84   85   86   87   88   89   90   91   92   93   94   95   96   97
   98   99  100  101  102  103  104  105  106  107  108  109  110  111
  112  113  114  115  116  117  118  119  120  121  122  123  124  125
  126  127  128  129  130  131  132  139  140  141  142  143  144  145
  146  147  148  149  150  151  152  153  154  155  156  157  158  159
  160  161  162  163  164  165  166  167  168  169  170  171  172  173
  174  175  176  191  192  193  194  195  196  197  198  199  200  201
  202  203  204  205  206  207  208  209  210  211  212  213  214  215
  216 

In [ ]:
x_new = smooth_mesh(v,f,handle_idx, new_pos) # smooth the mesh using the handle vertices

In [105]:
# plot the deformed mesh
p = mp.plot(v, f, c=labels, shading={"wireframe": True}) 
p = mp.plot(x, f, c=labels, shading={"wireframe": True})
p = mp.plot(x_new, f, c=labels, shading={"wireframe": True})

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.44…

## Step 2: Deforming the smooth mesh

The new deformed mesh is computed similarly to the previous step, by solving the minimization:
<img align="center" width="200" src="https://i.imgur.com/xv8ZcsA.png">
<!-- $$ 
\begin{aligned} \min_v& \quad v^T \textbf{L}_\omega \textbf{M}^{-1} \textbf{L}_\omega \textbf{v} \\
 \text{subject to}&
 \quad \textbf{v}_H = t(\textbf{o}_H),
\end{aligned}
$$ -->
where $t(\textbf{o}_H)$ are the new handle vertex positions after applying the user's transformation. We call this mesh $\mathcal{B}'$.

*Relevant `scipy` functions:* `scipy.sparse.linalg.spsolve` 

I'm still using my previous function for this

## Step 3: Encode Displacement Vectors (in local frame) 

<img align="left" width="200" src="img/hand_bd.png"> 
<img align="left" width="200" src="img/hand_td.png">
<br clear="both"/>
<img align="left" width="200" src="img/woody_bd.png">
<img align="left" width="200" src="img/woody_td.png">
<br clear="both"/>

*Fig 4: Displacements on $B$ (left) and $B'$ (right)*

The high-frequency details on the original surface are extracted from $\mathcal{S}$ and transferred to $\mathcal{B}'$. We first encode the high-frequency details of $\mathcal{S}$ as displacements w.r.t. $\mathcal{B}$.
We define an orthogonal reference frame on every vertex $v$ of $\mathcal{B}$ using:
1. The unit vertex normal
2. The normalized projection of one of $v$'s outgoing edges onto the tangent plane defined by the vertex normal. A stable choice is the edge whose projection onto the tangent plane is longest.
3. The cross-product between (1) and (2)

For every vertex $v$, we compute the displacement vector that takes $v$ from $\mathcal{B}$ to $\mathcal{S}$ and represent it as a vector in $v$'s reference frame. 
For every vertex of $\mathcal{B}'$, we also construct a reference frame using the normal and the SAME outgoing edge we selected for $\mathcal{B}$ (not the longest in $\mathcal{B}'$; it is important that the edges used to build both reference frames are the same). We can now use the displacement vector components computed in the previous paragraph to define transferred displacement vectors in the new reference frames of $\mathcal{B}'$. See Figure 4 for an example.
Applying the transferred displacements to the vertices of $\mathcal{B}'$ generates the final deformed mesh $\mathcal{S}'$. See Figure 5 for an example.

<img align="left" width="200" src="img/hand_f.png">
<img align="left" width="200" src="img/woody_f.png">
<br clear="both"/>

*Fig 5: Final Deformation Results*

Recommended outputs:
- Provide screenshots for 4 different deformed meshes. For each example, provide a rendering of $\mathcal{S}$, $\mathcal{B}$, $\mathcal{B}'$ and $\mathcal{S}'$.


In [106]:
""" 
We define an orthogonal reference frame on every vertex V of B using:
1. The unit vertex normal
2. The normalized projection of one of v outgoing edges onto the tangent plane defined by the vertex normal. 
    A stable choice is the edge whose projection onto the tangent plane is longest.
3. The cross-product between (1) and (2)
"""

B = x 
B_prime = x_new

""" Encoding details of B """
B_norms = igl.per_vertex_normals(B, f)
adjacent_vs = igl.adjacency_list(f)
detail_b = np.zeros_like(B)
dets = v-B
coeffs = np.zeros_like(dets) # coefficients of the reference frame
tangent_vectors = np.zeros_like(B) # Store the actual tangent vector
for idx, vertex in enumerate(B):
    # get the detail at this vertex
    di = dets[idx]
    # find the edge with the longest projection onto the tangent plane
    max_tangent_norm = 0
    best_tangent = np.zeros(3)
    for vert_idx in adjacent_vs[idx]:
        norm = B_norms[idx]
        edge = B[vert_idx] - vertex
        proj_normal = np.dot(edge, norm) * norm
        proj_tangent = edge - proj_normal
        tangent_norm = np.linalg.norm(proj_tangent)
        if tangent_norm > max_tangent_norm:
            max_tangent_norm = tangent_norm
            best_tangent = proj_tangent
    # get the vectors in our frame
    xi = best_tangent / np.linalg.norm(best_tangent) # normalized tangent
    # save the tangent for the B'
    tangent_vectors[idx] = xi # Store the tangent vector
    norm = B_norms[idx]
    yi = np.cross(norm, xi) # cross product
    # encode the detail -- details[idx] = di[0]*xi + di[1]*yi + di[2]*norm -> coeffs are di[0], di[1], di[2]
    coeffs[idx] = [np.dot(di, xi), np.dot(di, yi), np.dot(di, norm)]
        
Bp_norms = igl.per_vertex_normals(B_prime, f)
dis_p = np.zeros_like(dets)
for idx, vertex in enumerate(B_prime):
    coeff = coeffs[idx] # coefficients of the reference frame
    norm = Bp_norms[idx]
    xi_prime = tangent_vectors[idx] # using the same tangents as prev
    yi_prime = np.cross(norm, xi_prime)
    dis_p[idx] = coeff[0]*xi_prime + coeff[1]*yi_prime + coeff[2]*norm

S = B_prime + dis_p
# mp.plot(v, f, c=labels, shading={"wireframe": True}).add_lines(B, B+cff, shading={"line_color": "red", "line_width": 1})
mp.plot(B, f, c=labels, shading={"wireframe": True}) #.add_lines(B, B+dis, shading={"line_color": "red", "line_width": 1})
mp.plot(B_prime, f, c=labels, shading={"wireframe": True}) #.add_lines(B_prime, B_prime+dis_p, shading={"line_color": "red", "line_width": 1})
# mp.plot(B, f, c=labels, shading={"wireframe": True}).add_lines(B, B[edge_idx], shading={"line_color": "blue", "line_width": 1})
# mp.plot(B_prime, f, c=labels, shading={"wireframe": True}).add_lines(B_prime, B_prime[edge_idx], shading={"line_color": "blue", "line_width": 1})
mp.plot(S, f, c=labels, shading={"wireframe": True})



Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.44…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.44…

In [107]:
p4 = mp.plot(v, f, c=labels, shading={"wireframe": True})
p2 = mp.plot(B, f, c=labels, shading={"wireframe": True})
p1 = mp.plot(B_prime, f, c=labels, shading={"wireframe": True})
p3 = mp.plot(S, f, c=labels, shading={"wireframe": True})


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.19…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.44…

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5, 0.44…

## Extra-credit: 
To achieve real-time performance, you must prefactor the sparse bi-Laplacian matrix appearing in both linear systems. After the user specifies vertex sets $H$ and $F$, you can factorize the matrix $L_\omega M^{-1} L_\omega$ (using a Cholesky $LL^T$ factorization) and then re-use the factorization to solve both linear systems efficiently. This is an optional part of the exercise; if your implementation does not achieve interactive frame-rates (10+ fps) on the gingerbread mesh, it will not receive the full score. This might require additional vectorizations.

*Available Packages*: `scikit-sparse`, `numba`.
*Relevant functions*: `sksparse.cholmod`, `numba.jit`, `numpy.einsum`, `scipy.sparse.linalg.splu`